**Tugas**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan  untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca .

Prosedurnya adalah:

1. Jalankan Model dan hitung loss dengan .
2. Hitung update dan terapkan pada model dengan optimizer

---

**Praktikum 2**

---

In [1]:
import tensorflow as tf # digunakan untuk membangun dan melatih model machine learning
import numpy as np # digunakan untuk menangani array multidimensi dan operasi matematis
import os # digunakan untuk berinteraksi dengan sistem operasi
import time # digunakan untuk mengukur waktu

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt') # mengunduh file dari URL ke direktori cache Keras

1115394/1115394 [==============================] - 4s 4us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') # digunakan untuk membaca file teks yang dikodekan dalam UTF-8 dan mengembalikan konten file tersebut sebagai string
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters') # mencetak panjang dari string text

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250]) # Menampilkan 250 karakter pertama

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text)) # membuat vocab dengan set dari teks yang diurutkan
print(f'{len(vocab)} unique characters') # menampilkan panjang variabel vocab

65 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz'] # Membuat list example_texts
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # memecah teks yang diberikan menjadi urutan poin kode Unicode
chars # menampilkan variabel chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) # membuat lapisan StringLookup TensorFlow yang dapat digunakan untuk mengkonversi token teks menjadi ID numerik

In [8]:
ids = ids_from_chars(chars) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
ids # Menampilkan variabel ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) # membuat lapisan StringLookup TensorFlow dengan tambahan argument invert=True dan tidak menggunakan token mask

In [10]:
chars = chars_from_ids(ids) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
chars # Menampilkan variabel ids

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [103]:
tf.strings.reduce_join(chars, axis=-1).numpy() # menggabungkan nilai pada chars menjadi string

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # mengembalikan string yang digabungkan dari semua token teks dalam urutan ID numerik ids

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) # mengkonversi semua karakter dalam teks text menjadi ID numerik menggunakan lapisan StringLookup ids_from_chars
all_ids # Menamplikan variabel all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [13]:

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # membuat dataset TensorFlow dari urutan ID numerik all_ids

In [14]:
for ids in ids_dataset.take(10): # Melakukan perulangan pada 10 elemen pertama dari dataset ids_dataset
    print(chars_from_ids(ids).numpy().decode('utf-8')) #  mencetak token teks yang sesuai dengan ID numerik ids

F
i
r
s
t
 
C
i
t
i


In [15]:
seq_length = 100 # Inisialisasi variabel seq_length

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # membuat batch tensor ID numerik dari dataset ids_dataset, dengan ID urutan selanjutnya sebagai target dan mengabaikan batch terakhir jika panjangnya kurang dari seq_length+1

for seq in sequences.take(1): # Melakikan perulangan nilai pertama dari data sequence
  print(chars_from_ids(seq)) # mencetak token teks yang sesuai dengan urutan ID numerik seq

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5): # Melakukan perulangan pada 5 elemen pertama dari dataset ids_dataset
    print(text_from_ids(seq).numpy()) # mencetak teks yang sesuai dengan urutan ID numerik seq

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1] # mengambil seluruh nilai kecuali nilai terakhir
    target_text = sequence[1:] # mengambil seluruh nilai kecuali nilai pertama
    return input_text, target_text # mengembalikan variabel input_text dan target_text

In [19]:
split_input_target(list("Tensorflow")) # menjalankan function split_input_target dengan argument list 'Tensorflow'

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target) # memetakan setiap elemen dalam dataset sequences ke urutan ID numerik yang berisi input teks dan target teks

In [21]:
for input_example, target_example in dataset.take(1): # melakukan perulangan pada elemen pertama dataset dan membaginya ke variabel input_example dan target_example
    print("Input :", text_from_ids(input_example).numpy()) # menampilkan nilai input_example
    print("Target:", text_from_ids(target_example).numpy()) # menampilkan nilai target_example

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [22]:
# Batch size
BATCH_SIZE = 64 # Inisialisasi variabel BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000 # Inisialisasi variabel BUFFER_SIZE

dataset = (
    dataset # Membuat dataset
    .shuffle(BUFFER_SIZE) # mengacak urutan elemen dalam dataset dan menyimpan elemen-elemen yang diacak berdasarkan buffer_size
    .batch(BATCH_SIZE, drop_remainder=True) # mengelompokkan elemen-elemen dalam dataset menjadi batch dan  elemen-elemen yang tidak dapat dibagi oleh BATCH_SIZE akan dibuang
    .prefetch(tf.data.experimental.AUTOTUNE)) # TensorFlow secara otomatis menentukan ukuran buffer yang optimal untuk prefetch

dataset # menampilkan variabel dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary()) # menghitung ukuran vocabulary dari lapisan StringLookup ids_from_chars

# The embedding dimension
embedding_dim = 256 # Inisialisasi variabel embedding_dim

# Number of RNN units
rnn_units = 1024 # Inisialisasi variabel rnn_units

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # membuat embedding layer untuk memproses urutan ID numerik yang dihasilkan oleh lapisan StringLookup
    self.gru = tf.keras.layers.GRU(rnn_units, # Jumlah unit sebanyak rnn_units
                                   return_sequences=True, # mengembalikan urutan vektor numerik yang sama dengan panjang urutan input
                                   return_state=True) # membuat lapisan GRU untuk memproses urutan vektor numerik yang dihasilkan oleh lapisan embedding dan mengembalikan keadaan internal dari lapisan GRU
    self.dense = tf.keras.layers.Dense(vocab_size) # membuat layer dense dengan argument vacab_size

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs # menyimpan variabel inputs ke dalam variabel x
    x = self.embedding(x, training=training) # memproses urutan ID numerik x menggunakan lapisan embedding self.embedding dengan set mode training berdasarkan variabel training
    if states is None:
      states = self.gru.get_initial_state(x) # mendapatkan keadaan awal dari lapisan GRU
    x, states = self.gru(x, initial_state=states, training=training) # memproses urutan vektor numerik x menggunakan lapisan GRU self.gru dengan keadaan awal states dan mode berdasarkan variabel training
    x = self.dense(x, training=training) # memproses urutan vektor numerik x menggunakan lapisan dense

    if return_state:
      return x, states # mengembalikan nilai x dan states
    else:
      return x # hanya mengembalikan nilai x

In [25]:
model = MyModel( # Memanggil function MyModel()
    vocab_size=vocab_size, # menambahkan argument berupa variabel vocab_size
    embedding_dim=embedding_dim, # menambahkan argument berupa variabel embedding_dim
    rnn_units=rnn_units) # menambahkan argument berupa variabel rnn_units

In [26]:
for input_example_batch, target_example_batch in dataset.take(1): # Melukan perulangan dengan elemen pertama yang dibagi dalam variabel input_example_batch dan target_example_batch
    example_batch_predictions = model(input_example_batch) # menghasilkan prediksi untuk batch input input_example_batch menggunakan model
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary() # mencetak ringkasan dari model machine learning yang telah dibuat

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # mengambil sampel dari distribusi kategorikal yang ditentukan oleh vektor probabilitas example_batch_predictions[0] dengan jumlah sampel yang diambil adalah 1
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # menghapus dimensi dari tensor dan mengubah tensor sampled_indices menjadi array NumPy

In [29]:
sampled_indices # menampilkan array variabel sampled_indices

array([47, 57, 57, 52, 65, 26, 37, 28, 62, 26, 26, 46, 47, 25, 33, 43, 36,
       35, 10,  8, 53, 48, 61, 14, 64, 57,  1, 42, 49, 31,  2, 64,  1, 22,
       65, 46,  2, 27, 10, 34, 12, 40, 10, 57, 41,  8, 12, 55, 60, 54, 38,
       34, 45, 65, 40, 36, 44, 64, 57, 65, 42, 38, 23,  2,  0,  8,  6, 20,
       20, 10,  6, 60, 63, 36, 11, 18,  2, 34, 20, 63, 48, 49, 18, 44, 57,
       50, 39, 17,  3, 50, 65, 58, 27,  5, 46, 36, 20, 43, 42,  9],
      dtype=int64)

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) # mencetak input pertama dalam batch input_example_batch
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy()) # mencetak prediksi karakter berikutnya untuk input pertama dalam batch input_example_batch

Input:
 b'orth that fatal screech-owl to our house,\nThat nothing sung but death to us and ours:\nNow death shal'

Next Char Predictions:
 b"hrrmzMXOwMMghLTdWV3-nivAyr\ncjR y\nIzg N3U;a3rb-;puoYUfzaWeyrzcYJ [UNK]-'GG3'uxW:E UGxijEerkZD!kzsN&gWGdc."


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # membuat fungsi kerugian Sparse Categorical Crossentropy dengan menerima logits sebagai input

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions) # menghitung rata-rata kerugian untuk batch input example_batch_predictions
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions
print("Mean loss:        ", example_batch_mean_loss) # menampilkan mean loss

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1885705, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy() # menghitung eksponen dari rata-rata kerugian untuk batch input

65.92848

In [34]:
model.compile(optimizer='adam', loss=loss) # Compile model denga optimizer ADAM dan argument variabel loss

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints' # membuat direktori untuk menyimpan checkpoint model
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # membuat prefix nama file untuk checkpoint model

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True) # menyimpan checkpoint model machine learning ke direktori checkpoint_dir dengan prefix nama file ckpt_{epoch} dan yang disimpan hanya berupa bobotnya

In [36]:
EPOCHS = 20 # Inisialisasi variabel EPOCHS

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) # melatih model  dengan dataset dataset selama sebanyak EPOCHS, dengan menggunakan callback untuk menyimpan checkpoint model

Epoch 1/20


172/172 [==============================] - 942s 5s/step - loss: 2.7123
Epoch 2/20
100/172 [================>.............] - ETA: 6:50 - loss: 2.0575

---

**Praktikum Tugas**

---

chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) # membuat lapisan StringLookup TensorFlow dengan tambahan argument invert=True dan tidak menggunakan token mask

In [130]:
class CustomTraining(MyModel):
  @tf.function # mengonversi fungsi Python menjadi graph TensorFlow
  def train_step(self, inputs):
      inputs, labels = inputs # Menyimpan value inputs dengan membaginya ke inputs dan labels
      with tf.GradientTape() as tape: # Membuat GradientTape untuk menghitung gradien dari operasi TensorFlow yang direkam
          predictions = self(inputs, training=True) # memanggil model untuk menghasilkan prediksi dengan mengaktifkan mode training
          loss = self.loss(labels, predictions) # menghitung kerugian model dengan membandingkan labels dengan predictions

      grads = tape.gradient(loss, model.trainable_variables) # menghitung gradien dari kerugian model terhadap variabel-variabel yang dapat dilatih
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables)) # memperbarui parameter model menggunakan optimizer

      return {'loss': loss} # Mengembalikan dictionary dengan key 'loss' dan value variabel loss

Kode diatas menerapkan train_step method sesuai dengan  Keras' train_step conventions. Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras Model.compile and Model.fit methods.

In [131]:
model = CustomTraining( # Membuat Model CustomTraining
    vocab_size=len(ids_from_chars.get_vocabulary()), # dengan kosakata yang digunakna sebanyak vocabulary yang dibuat sebelumnya
    embedding_dim=embedding_dim, # argument embedding_dim dari variabel embedding_dim
    rnn_units=rnn_units) # dan banyak unit rnn dengan variabel rnn_units

In [132]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)) # Compile model dengan optimizer ADAM yang menhasilkan output berupa logits dengan menggunakan function loss berupa SparseCategoricalCrossentropy

In [133]:
model.fit(dataset, epochs=1) # Melatih model dengan dataset dan 1 epochs

172/172 [==============================] - 14s 57ms/step - loss: 2.7443


Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri:

In [134]:
EPOCHS = 20 # Inisisalisasi variabel EPOCH

mean = tf.metrics.Mean() # membuat metrik rata-rata

for epoch in range(EPOCHS): # Perulangan sebanyak EPCOHS
    start = time.time() # Mencatat waktu mulai proses pada perulangan saat ini

    mean.reset_states() # mereset metrik rata-rata
    for (batch_n, (inp, target)) in enumerate(dataset): # melakukan iterasi pada dataset dan mendapatkan batch data yang terdiri dari input dan target dengan menambahkan indeks disetiap elemennya
        logs = model.train_step([inp, target]) # melatih modelg dengan menggunakan batch data yang diberikan
        mean.update_state(logs['loss']) # memperbarui metrik rata-rata dengan loss training

        if batch_n % 50 == 0: # Jika batch_n modulus 50 sama dengan 0
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}" # Menampilkan Epoch saat ini, value batch dan loss
            print(template) # Menampilkan template

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0: # Jika epoch + 1 modulus 5 sama dengan 0
        model.save_weights(checkpoint_prefix.format(epoch=epoch)) # menyimpan bobot model ke file dengan memanfaatkan checkpoint_prefix sebelumnya

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}') # Menampilkan epoch saat ini dan hasil dari mean
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec') # Menampilkan estimasi waktu dari selisih waktu saat ini dengan start
    print("_"*80) # Menampilkan 80 garis bawah

model.save_weights(checkpoint_prefix.format(epoch=epoch)) # menyimpan bobot model machine learning ke file dengan memanfaatkan checkpoint_prefix sebelumnya

Epoch 1 Batch 0 Loss 2.1690
Epoch 1 Batch 50 Loss 2.0413
Epoch 1 Batch 100 Loss 1.9430
Epoch 1 Batch 150 Loss 1.8204

Epoch 1 Loss: 2.0049
Time taken for 1 epoch 13.63 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8044
Epoch 2 Batch 50 Loss 1.7919
Epoch 2 Batch 100 Loss 1.7259
Epoch 2 Batch 150 Loss 1.6558

Epoch 2 Loss: 1.7305
Time taken for 1 epoch 12.99 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6224
Epoch 3 Batch 50 Loss 1.5703
Epoch 3 Batch 100 Loss 1.5607
Epoch 3 Batch 150 Loss 1.5154

Epoch 3 Loss: 1.5678
Time taken for 1 epoch 12.43 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4588
Epoch 4 Batch 50 Loss 1.4705
Epoch 4 Batch 100 Loss 1.4582
Epoch 4 Batch 150 Loss 1.4159

Epoch 4 Loss: 1.4667
Time taken for 1 epoch 11.96 sec
_____________________________________________________________________

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

- Pada **Praktikum 2** memiliki estimasi waktu yang lebih cepat untuk menjalankan proses disetiap epoch daripada **Praktikum Tugas**, dikarenakan pada **Praktikum Tugas** walaupun estimasi waktunya masih serupa dengan **Praktikum 2** yaitu berkisar antara 11s sampai 14s, namun pada **Praktikum Tugas** terjadi lonjakan estimasi waktu pada epoch 15 mencapai 12.6s.

- Namun terjadi hal sebaliknya pada nilai loss yang dihasilkan, pada **Praktikum Tugas** memiliki nilai loss yang lebih sedikit daripada pada **Praktikum 2**, jika pada **Praktikum 2** nilai loss pada epoch satu adalah 2.7240 dan pada epoch dua puluh menghasilkan nilai loss 0.7258 sedangkan pada **Praktikum Tugas** dimulai dengan nilai loss sebesar 2.0049 pada epoch satu dan diakhiri dengan nilai loss sebesar 0.7067 pada epoch kedua puluh